# Data Importing

In [1]:
import pandas as pd
#C://Users//bijoy//Dropbox//PC//Downloads//GHG_SouthAsia//test2.xlsx
#df = pd.read_excel("C:/Users/Bijoy/Downloads/airpoltest/Final/Points/new.xlsx")
#df= df.drop(['Zone'],axis=1)
df = pd.read_excel("E://BIJOY_GIS//project_air_pol//Global_PM2.5//!CompleteTrain.xlsx")
df= df.drop(['Region','FID','Id','PM25_org'],axis=1)
df#.describe()

,PM25,X,Y,NDVI,NDBI,MNDWI,DEM,NTL,LST
0,1.760455,43.075696,10.083458,0.233707,0.103238,-0.441356,1484,0.161978,310.365438
1,1.760455,43.527811,10.083458,0.237938,0.140716,-0.451067,1413,0.158229,310.041212
2,1.801270,43.979926,10.083458,0.219778,0.117881,-0.403485,759,0.167162,312.169284
3,1.841122,44.432040,10.083458,0.130274,0.139700,-0.477442,496,0.186774,315.994085
4,1.818728,44.884155,10.083458,0.126678,0.116530,-0.372730,643,0.191910,315.389267
...,...,...,...,...,...,...,...,...,...
49912,1.523277,-55.111800,5.678925,0.642777,-0.301792,-0.338804,11,0.883484,304.668871
49913,1.523277,-54.784319,5.678925,0.593957,-0.298592,-0.286398,20,0.187292,303.990573
49914,1.522663,-54.456838,5.678925,0.571804,-0.296414,-0.263144,16,0.188188,304.388080
49915,1.522663,-54.129357,5.678925,0.505683,-0.266910,-0.216042,12,0.184743,305.365465


In [2]:
df.corr()

,PM25,X,Y,NDVI,NDBI,MNDWI,DEM,NTL,LST
PM25,1.000000,0.411197,-0.101264,-0.181930,0.582875,-0.409349,-0.268845,-0.003571,0.624833
X,0.411197,1.000000,0.233582,-0.192025,0.122732,0.023520,0.060472,0.000962,-0.024498
Y,-0.101264,0.233582,1.000000,-0.512782,-0.314670,0.627547,0.020872,0.040096,-0.586465
NDVI,-0.181930,-0.192025,-0.512782,1.000000,-0.190217,-0.507543,-0.284587,-0.011125,0.144134
NDBI,0.582875,0.122732,-0.314670,-0.190217,1.000000,-0.734520,0.043823,-0.000308,0.789761
MNDWI,-0.409349,0.023520,0.627547,-0.507543,-0.734520,1.000000,0.146391,0.012456,-0.801310
DEM,-0.268845,0.060472,0.020872,-0.284587,0.043823,0.146391,1.000000,-0.040208,-0.180883
NTL,-0.003571,0.000962,0.040096,-0.011125,-0.000308,0.012456,-0.040208,1.000000,-0.009532
LST,0.624833,-0.024498,-0.586465,0.144134,0.789761,-0.801310,-0.180883,-0.009532,1.000000


# Incorporating a 1D CNN model to predict log(PM2.5)

Data importing

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten


# Load your dataset
df = df
# Define features and target variable
X = df.drop(['PM25','X','Y'],axis=1)
y = df['PM25']

# Splitting dataset into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for CNN
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Scaling features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)


Model Building

In [4]:
# Defining the CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1)
])

# Compiling the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Fitting the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

C:\Users\Bijoy\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.3288 - mae: 0.4157 - val_loss: 0.0446 - val_mae: 0.1600
Epoch 2/50
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0870 - mae: 0.2304 - val_loss: 0.0433 - val_mae: 0.1566
Epoch 3/50
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0600 - mae: 0.1896 - val_loss: 0.0368 - val_mae: 0.1453
Epoch 4/50
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0464 - mae: 0.1648 - val_loss: 0.0357 - val_mae: 0.1445
Epoch 5/50
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0417 - mae: 0.1563 - val_loss: 0.0359 - val_mae: 0.1445
Epoch 6/50
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0394 - mae: 0.1521 - val_loss: 0.0331 - val_mae: 0.1350
Epoch 7/50
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0387 - mae: 0.1491 - val_loss: 0.0349 - val_mae: 0.1407
Epoch 8/50
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0376 - mae: 0.1481 - val_loss: 0.0320 - val_mae: 0.1343
Epoch 9/50
1248/1248 ━━━━━━━━━━━━━━━━━━━

Test Score Evaluation

In [5]:
# Making predictions on training data
y_train_pred = model.predict(X_train)

# Calculate RMSE for training predictions
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Calculate R-squared for training predictions
r2_train = r2_score(y_train, y_train_pred)

print(f"Root Mean Squared Error (Training): {rmse_train}")
print(f"R-squared (Training): {r2_train}")

# Making predictions on testing data
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")


1248/1248 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Root Mean Squared Error (Training): 0.1612554831594772
R-squared (Training): 0.7538013955065358
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Root Mean Squared Error: 0.16498309555572616
R-squared: 0.743600509737256
